In [1]:
import regex as re
from tqdm import tqdm

from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle

from paper_tree import PaperTree

import os
import torch
torch.cuda.empty_cache()

In [2]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device="cuda")

Device set to use cuda


In [3]:
def clean_junk(tex):
    # LHCb junk
    pattern = r"\\centerline[\n\s]*\{[\n\s]*\\large[\n\s]*\\bf[\n\s]*LHCb[\n\s]*collaboration[\n\s]*\}[\n\s]*\\begin[\n\s]*\{[\n\s]*flushleft[\n\s]*\}(?:\n|.)*\{[\n\s]*\\footnotesize(?:\n|.)*\}[\n\s]*\\end[\n\s]*\{[\n\s]*flushleft[\n\s]*\}"
    tex = re.sub(pattern, "", tex)

    # Remove ~ symbol
    pattern = r"\~"
    tex = re.sub(pattern, " ", tex)

    # LHCb junk
    pattern = r"\\bibitem\{.+\}(?:.|\n)*\\EndOfBibitem"
    tex = re.sub(pattern, "", tex)

    # Don't try to RAG on figures
    pattern = r"\\begin[\s\n]*\{[\s\n]*figure[\s\n]*\}(?:.|\n)*\\end[\s\n]*\{[\s\n]*figure[\s\n]*\}"
    tex = re.sub(pattern, "", tex)

    # Don't try to RAG on tables
    pattern = r"\\begin[\s\n]*\{[\s\n]*table[\s\n]*\}(?:.|\n)*\\end[\s\n]*\{[\s\n]*table[\s\n]*\}"
    tex = re.sub(pattern, "", tex)

    # Don't try to RAG on arrays
    pattern = r"\\begin[\s\n]*\{[\s\n]*array[\s\n]*\}(?:.|\n)*\\end[\s\n]*\{[\s\n]*array[\s\n]*\}"
    tex = re.sub(pattern, "", tex)

    # Don't try to RAG on tabular data
    pattern = r"\\begin[\s\n]*\{[\s\n]*tabular[\s\n]*\}(?:.|\n)*\\end[\s\n]*\{[\s\n]*tabular[\s\n]*\}"
    tex = re.sub(pattern, "", tex)

    # Don't try to RAG on citations
    pattern = r"\\cite\s*({(?:[^{}]*+|(?1))*})"
    tex = re.sub(pattern, "", tex)

    # Don't try to RAG on refs
    pattern = r"\\ref\s*({(?:[^{}]*+|(?1))*})"
    tex = re.sub(pattern, "", tex)
    return tex 

In [4]:
dir_expanded_tex = "../data/expanded_tex/"
dir_abstracts = "../data/abstracts/"
dir_paper_trees = "../data/paper_trees/"

os.makedirs(dir_paper_trees, exist_ok=True)
def tex_to_paper_tree(filename):
    with open(dir_abstracts + filename) as file:
        abstract = file.read()
    with open(dir_expanded_tex + filename) as file:
        full_tex = file.read()
    full_tex = clean_junk(full_tex)
    paper = PaperTree(filename, full_tex, abstract = abstract)
    return paper

filenames = list(filter(lambda str : (".tex" in str[-4:]), os.listdir(dir_expanded_tex)))

papers = []
times = []
with ThreadPoolExecutor(max_workers = 16) as executor:
    futures = [executor.submit(tex_to_paper_tree, filename) for filename in filenames]
    
    # Use tqdm to track progress as tasks complete
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
        paper = future.result()
        papers.append(paper)

Processing: 100%|██████████| 781/781 [00:13<00:00, 57.96it/s]


In [8]:
for paper in papers:
    if len(paper.sections) < 2:
        print(paper)
        break

1506.00903.tex



In [11]:
paper.text

'\n\n \n\n\n\n\\section*{Supplemental Material}\n\n\n\n\n\n\n\n\n'

In [5]:
needs_abstract = []

def build_needs_abstract(paper, needs_abstract):
    for section in paper.sections:
        if len(section.sections) > 0 and section.abstract is None:
            needs_abstract.append(section)
        build_needs_abstract(section, needs_abstract)

for paper in papers:
    build_needs_abstract(paper, needs_abstract)

len(needs_abstract)

2255

In [6]:
from transformers import BartTokenizer
import math

MAX_SUMMARY_LENGTH = 150
BART_MAX_TOKENS = 1024

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
# Function to split the text into chunks with overlap
def split_into_chunks(text, max_length = BART_MAX_TOKENS - MAX_SUMMARY_LENGTH):
    # Tokenize the text into tokens
    tokens = tokenizer.encode(text, truncation=False)

    # Calculate the ideal chunk size (as equal as possible)
    chunk_size = math.ceil(len(tokens) / math.ceil(len(tokens) / max_length))

    # Calculate how many chunks are needed
    num_chunks = math.ceil(len(tokens) / chunk_size)

    chunks = []
    start_idx = 0

    for _ in range(num_chunks):
        end_idx = start_idx + chunk_size
        chunk = tokens[start_idx:end_idx]
        chunks.append(chunk)

        # Update start_idx for the next chunk
        start_idx = end_idx

    # Decode chunks back to text
    chunk_texts = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

    return chunk_texts, len(tokens)


In [54]:
for section in tqdm(needs_abstract, desc="Processing sections", unit="section"):
    # Dont try to summarize Headers, its just a bunch of LaTeX junk
    if section.title == "Headers":
        continue
    # Don't re-generate an abstract
    if section.abstract is not None:
        continue
    location = section.title
    parent = section.parent
    while not (parent is None):
        location = f"{parent.title} --> {location}"
        parent = parent.parent

    string = section.text
    chunked_string, num_tokens = split_into_chunks(string)
    # The text excerpt is too small to justify generating an abstract
    if num_tokens < 300:
        continue
    
    tqdm.write(f"Processing section: {location} | Number of chunks: {len(chunked_string)}")
    
    summaries = summarizer(chunked_string, max_length = max(MAX_SUMMARY_LENGTH // len(chunked_string), 40), min_length = 5, length_penalty = 5.0, do_sample = True, temperature = 0.25)
    section.abstract = "".join([summary['summary_text'] + " " for summary in summaries])

Processing sections:   0%|          | 0/2255 [00:00<?, ?section/s]

Processing section: 1112.4698.tex --> Determination of proper-time acceptance effects | Number of chunks: 2


Processing sections:  13%|█▎        | 301/2255 [00:01<00:06, 303.04section/s]

Processing section: 1610.01383.tex --> Detector and simulation | Number of chunks: 1


Processing sections:  13%|█▎        | 301/2255 [00:01<00:06, 303.04section/s]

...

Processing sections: 100%|█████████▉| 2253/2255 [27:02<00:01,  1.24section/s]

Processing section: 1308.4583.tex --> Systematic uncertainties | Number of chunks: 1


Processing sections: 100%|█████████▉| 2254/2255 [27:03<00:00,  1.42section/s]

Processing section: 1308.4583.tex --> Results and conclusions | Number of chunks: 2


Processing sections: 100%|██████████| 2255/2255 [27:03<00:00,  1.39section/s]


In [19]:
needs_abstract[i].abstract = None

In [52]:
i = 229
print(needs_abstract[i].title)

tex = needs_abstract[i].text
print(tex)

Signal selection and data analysis

\label{sec:ana}

The signal selection is largely performed at the trigger level. 
The offline analysis, in addition, requires the transverse momentum of \proton and \antiproton to be 
$\pt > 2.0 \gevc$, and restricts charmonium candidates to the rapidity range $2.0 < y < 4.5$. 

Discrimination between promptly produced charmonium candidates and those from 
$b$-hadron decays is achieved using the pseudo-decay time $t_{z} = \Delta z \cdot M / p_z$, 
where $\Delta z$ is the distance along the beam axis between 
the corresponding \proton\proton collision vertex (primary vertex) 
and the candidate decay vertex, $M$ is the candidate mass, and $p_z$ is the longitudinal component 
of its momentum. 
Candidates with $t_{z} < 80 \fs$ are classified as prompt, while those 
with $t_{z} > 80 \fs$ are classified as having originated from \bquark-hadron decays. 
For charmonium candidates from \bquark-hadron decays, a significant displacement 
of the proton tracks wi

In [51]:
print(needs_abstract[i].abstract)

We first repeat all the fits, including the spin-parity analysis, lowering the \pt requirement from 7.5 to 7.0 GeV. We find that all the resonance parameters vary within their statistical uncertainties. The background lineshapes are additionaly tested by random variation of their parameters. For each mass spectrum, we generate and fit 500 new histograms where the. resonance parameters and yields are fixed to the values obtained from the data. 


In [55]:
os.makedirs(dir_paper_trees, exist_ok = True)

for paper in papers:
    with open(dir_paper_trees + paper.title[:-4] + ".pkl", "wb") as f:
        pickle.dump(paper, f)
